In [58]:
## Import Essential Data handling libraries
import pandas as pd
import numpy as np

from math import floor
from time import perf_counter

import matplotlib.pyplot as plt,seaborn as sns
import matplotlib as mpl
%matplotlib inline


from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

In [59]:
### Importing Machine Learning Libraries and functions
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

from sklearn.metrics import precision_recall_curve, auc, f1_score
from sklearn.metrics import make_scorer


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from xgboost import XGBClassifier

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import RFECV

In [84]:
from functools import partial
from skopt import space
from skopt import gp_minimize

from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope


import optuna

#### Loading Data

In [61]:
df_raw = pd.read_csv("..\data\creditcard.csv")

In [62]:
df_raw.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000,-1.360,-0.073,2.536,1.378,-0.338,0.462,0.240,0.099,0.364,0.091,-0.552,-0.618,-0.991,-0.311,1.468,-0.470,0.208,0.026,0.404,0.251,-0.018,0.278,-0.110,0.067,0.129,-0.189,0.134,-0.021,149.620,0
1,0.000,1.192,0.266,0.166,0.448,0.060,-0.082,-0.079,0.085,-0.255,-0.167,1.613,1.065,0.489,-0.144,0.636,0.464,-0.115,-0.183,-0.146,-0.069,-0.226,-0.639,0.101,-0.340,0.167,0.126,-0.009,0.015,2.690,0
2,1.000,-1.358,-1.340,1.773,0.380,-0.503,1.800,0.791,0.248,-1.515,0.208,0.625,0.066,0.717,-0.166,2.346,-2.890,1.110,-0.121,-2.262,0.525,0.248,0.772,0.909,-0.689,-0.328,-0.139,-0.055,-0.060,378.660,0
3,1.000,-0.966,-0.185,1.793,-0.863,-0.010,1.247,0.238,0.377,-1.387,-0.055,-0.226,0.178,0.508,-0.288,-0.631,-1.060,-0.684,1.966,-1.233,-0.208,-0.108,0.005,-0.190,-1.176,0.647,-0.222,0.063,0.061,123.500,0
4,2.000,-1.158,0.878,1.549,0.403,-0.407,0.096,0.593,-0.271,0.818,0.753,-0.823,0.538,1.346,-1.120,0.175,-0.451,-0.237,-0.038,0.803,0.409,-0.009,0.798,-0.137,0.141,-0.206,0.502,0.219,0.215,69.990,0


In [63]:
df_raw.drop(["Time"], axis = 1, inplace  = True)

##### Checking Class Distribution

In [64]:
print(df_raw.Class.value_counts())

print("\n","% distribution\n",df_raw.Class.value_counts(True)*100)

0    284315
1       492
Name: Class, dtype: int64

 % distribution
 0   99.827
1    0.173
Name: Class, dtype: float64


In [65]:
df_raw.shape

(284807, 30)

#### Scoring function

In [66]:
def AUPRC(y_true, y_pred_proba) :
    precision, recall, thresholds = precision_recall_curve(y_true = y_true, probas_pred= y_pred_proba)
    auprc = auc(recall, precision)
    return(np.round(auprc, 4))

#### Data Transformation
* 1. Quantile Transformation (normal distribution)
* 2. standard scaling the data

In [67]:
quantile_transformer = QuantileTransformer(output_distribution="normal")
standard_scaler = StandardScaler()

In [68]:
df_transformed = df_raw.copy()

for i in range(df_transformed.iloc[:,:-1].shape[1]):
    qt_transformed_var = quantile_transformer.fit_transform(df_transformed.iloc[:,i].values.reshape(-1,1))[:,0]
    qt_transformed_std_scaled = standard_scaler.fit_transform(qt_transformed_var.reshape(-1,1))[:,0]
    
    df_transformed.iloc[:,i] = qt_transformed_std_scaled

In [69]:
df_transformed.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.999,-0.158,2.132,1.191,-0.304,0.725,0.287,0.200,0.438,0.307,-0.465,-0.885,-1.001,-0.519,1.771,-0.677,0.399,0.043,0.598,0.966,0.036,0.350,-0.459,0.079,0.196,-0.308,0.836,-0.320,1.030,0
1,0.459,0.216,-0.007,0.417,0.122,0.217,-0.160,0.164,-0.249,-0.117,1.637,1.359,0.509,-0.287,0.664,0.582,-0.071,-0.250,-0.224,-0.027,-0.664,-0.837,0.491,-0.650,0.270,0.389,-0.094,0.055,-0.838,0
2,-0.998,-1.270,1.382,0.344,-0.486,1.374,0.962,0.536,-1.456,0.450,0.571,-0.095,0.730,-0.318,2.777,-2.673,1.488,-0.162,-2.491,1.379,0.886,1.036,2.020,-1.121,-0.697,-0.193,-0.535,-0.766,1.596,0
3,-0.718,-0.285,1.400,-0.689,0.047,1.185,0.284,0.760,-1.359,0.062,-0.171,0.050,0.528,-0.487,-0.722,-1.282,-1.049,2.194,-1.520,-0.657,-0.263,-0.003,-0.791,-1.675,1.393,-0.393,0.521,0.569,0.925,0
4,-0.861,0.750,1.160,0.368,-0.381,0.404,0.704,-0.855,0.864,0.953,-0.741,0.556,1.387,-1.385,0.145,-0.653,-0.254,-0.045,1.105,1.237,0.064,1.082,-0.575,0.224,-0.407,1.024,1.071,1.454,0.597,0


In [70]:
#selected vars (manual feature selection)
all_vars = list(set(list(df_transformed.columns)) - set(["V8","V13","V15", "V19", "V20", "V22", "V23","V24", "V25", "V26", "Amount"]))
 
df = df_transformed[all_vars]

In [71]:
df.head()

,V4,V12,V7,V18,V2,V14,V5,V9,V27,V16,V1,V3,V21,V11,Class,V17,V6,V10,V28
0,1.191,-0.885,0.287,0.043,-0.158,-0.519,-0.304,0.438,0.836,-0.677,-0.999,2.132,0.036,-0.465,0,0.399,0.725,0.307,-0.320
1,0.417,1.359,-0.160,-0.250,0.216,-0.287,0.122,-0.249,-0.094,0.582,0.459,-0.007,-0.664,1.637,0,-0.071,0.217,-0.117,0.055
2,0.344,-0.095,0.962,-0.162,-1.270,-0.318,-0.486,-1.456,-0.535,-2.673,-0.998,1.382,0.886,0.571,0,1.488,1.374,0.450,-0.766
3,-0.689,0.050,0.284,2.194,-0.285,-0.487,0.047,-1.359,0.521,-1.282,-0.718,1.400,-0.263,-0.171,0,-1.049,1.185,0.062,0.569
4,0.368,0.556,0.704,-0.045,0.750,-1.385,-0.381,0.864,1.071,-0.653,-0.861,1.160,0.064,-0.741,0,-0.254,0.404,0.953,1.454


#### HyperParameter Optimization

#### 1. HyperOpt

In [72]:
#### Scoring function
def AUPRC(y_true, y_pred_proba) :
    precision, recall, thresholds = precision_recall_curve(y_true = y_true, probas_pred= y_pred_proba)
    auprc = auc(recall, precision)
    return(np.round(auprc, 4))

In [73]:
def optimize(params, x, y):
    model = ExtraTreesClassifier(**params)
    kf = model_selection.StratifiedKFold(n_splits = 5)
    auprc = []
    for idx in kf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]
        
        xtest = x[test_idx]
        ytest = y[test_idx]
        
        model.fit(xtrain, ytrain)
        pred_proba = model.predict_proba(xtest)[:, 1]
        fold_auprc = AUPRC(ytest, pred_proba)
        
        auprc.append(fold_auprc)
    
    return -1.0*np.round(np.mean(auprc),4)

In [76]:
X = df.drop("Class", axis = 1).values
y = df.Class.values

param_space for ExtraTree
param_space = { 
    "n_estimators" : hp.choice("n_estimators", [100,150,200,250,300]),
    "max_depth" : hp.choice("max_depth", [3,5,7,9,11]),
    "criterion" : hp.choice("criterion", ["gini", "entropy"]),
    "min_samples_split" : hp.choice("min_samples_split", [2,4,6,8]),
    "min_samples_leaf": hp.choice("min_samples_leaf", [3,4,5]),
    "max_features": hp.choice("max_features", [4,5,6,7]),
    "n_jobs" : hp.choice("n_jobs", [-1])
}


optimization_function = partial(optimize, x = X, y = y)

trials = Trials()

result = fmin(
    algo = tpe.suggest,
    fn = optimization_function,
    space = param_space, 
    max_evals = 20,
    trials = trials
)

print(result)

100%|██████████████████████████████████████████████████████████| 20/20 [34:44<00:00, 104.22s/trial, best loss: -0.7753]
{'criterion': 0, 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 0, 'n_estimators': 0, 'n_jobs': 0}


In [83]:
#### Fitting on the above hyper parameters

In [86]:
model = ExtraTreesClassifier(max_depth=11, max_features=5, min_samples_leaf=4, min_samples_split=2, n_estimators=100, n_jobs = -1)
# model = ExtraTreesClassifier()

def create_stratified_folds(data, k_fold_num, target_variable):
    data["kfold"] = -1 # we create a new column called kfold and fill it with -1
    data = data.sample(frac=1).reset_index(drop=True) # the next step is to randomize the rows of the data

    y = data[target_variable].values

    kf = model_selection.StratifiedKFold(n_splits= k_fold_num)

    for fold, (trn_, partitioned_idx_) in enumerate(kf.split(X=data, y = y)):
        data.loc[partitioned_idx_, 'kfold'] = fold
    return(data)

# Creating stratified k-fold within the data
k_fold_num = 5

df = create_stratified_folds(df, k_fold_num = 5, target_variable = "Class")

features = [f for f in df.columns if f not in ["kfold","Class"]]

cv_auprc = []
f1_scores = []
for fold in range(k_fold_num):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    x_train = df_train[features].values
    x_valid = df_valid[features].values

    model.fit(x_train, df_train.Class.values)

    valid_preds = model.predict_proba(x_valid)[:, 1]
    valid_preds_binary = model.predict(x_valid)

    auprc = AUPRC(df_valid.Class.values, valid_preds)
    f1 = f1_score(df_valid.Class.values, valid_preds_binary)

    cv_auprc.append(auprc)
    f1_scores.append(np.round(f1,4))

print("Average Area Under Precision Recall Curve = ", np.round(np.mean(cv_auprc),2))
print("Average F1 score = ", np.round(np.mean(f1_scores),2))

Average Area Under Precision Recall Curve =  0.84
Average F1 score =  0.83


In [87]:
cv_auprc

[0.777, 0.8445, 0.8665, 0.8526, 0.845]

In [88]:
f1_scores

[0.7684, 0.8444, 0.8556, 0.8495, 0.8449]

#### with default hyperParameters

In [93]:
# model = ExtraTreesClassifier(max_depth=11, max_features=5, min_samples_leaf=4, min_samples_split=2, n_estimators=100, n_jobs = -1)
model = ExtraTreesClassifier()

def create_stratified_folds(data, k_fold_num, target_variable):
    data["kfold"] = -1 # we create a new column called kfold and fill it with -1
    data = data.sample(frac=1).reset_index(drop=True) # the next step is to randomize the rows of the data

    y = data[target_variable].values

    kf = model_selection.StratifiedKFold(n_splits= k_fold_num)

    for fold, (trn_, partitioned_idx_) in enumerate(kf.split(X=data, y = y)):
        data.loc[partitioned_idx_, 'kfold'] = fold
    return(data)

# Creating stratified k-fold within the data
k_fold_num = 5

df = create_stratified_folds(df, k_fold_num = 5, target_variable = "Class")

features = [f for f in df.columns if f not in ["kfold","Class"]]

cv_auprc = []
f1_scores = []
for fold in range(k_fold_num):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    x_train = df_train[features].values
    x_valid = df_valid[features].values

    model.fit(x_train, df_train.Class.values)

    valid_preds = model.predict_proba(x_valid)[:, 1]
    valid_preds_binary = model.predict(x_valid)

    auprc = AUPRC(df_valid.Class.values, valid_preds)
    f1 = f1_score(df_valid.Class.values, valid_preds_binary)

    cv_auprc.append(auprc)
    f1_scores.append(np.round(f1,4))

print("Average Area Under Precision Recall Curve = ", np.round(np.mean(cv_auprc),2))
print("Average F1 score = ", np.round(np.mean(f1_scores),2))

Average Area Under Precision Recall Curve =  0.85
Average F1 score =  0.86


In [94]:
cv_auprc

[0.8389, 0.8634, 0.8349, 0.8636, 0.8734]

In [95]:
f1_scores

[0.8539, 0.8889, 0.8492, 0.8743, 0.8539]

#### 2. Optuna

In [73]:
def optimize(trial, x, y):
    
    model = ExtraTreesClassifier(**params)
    kf = model_selection.StratifiedKFold(n_splits = 5)
    auprc = []
    for idx in kf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]
        
        xtest = x[test_idx]
        ytest = y[test_idx]
        
        model.fit(xtrain, ytrain)
        pred_proba = model.predict_proba(xtest)[:, 1]
        fold_auprc = AUPRC(ytest, pred_proba)
        
        auprc.append(fold_auprc)
    
    return -1.0*np.round(np.mean(auprc),4)

In [76]:
X = df.drop("Class", axis = 1).values
y = df.Class.values

# param_space for ExtraTree
param_space = { 
    "n_estimators" : hp.choice("n_estimators", [100,150,200,250,300]),
    "max_depth" : hp.choice("max_depth", [3,5,7,9,11]),
    "criterion" : hp.choice("criterion", ["gini", "entropy"]),
    "min_samples_split" : hp.choice("min_samples_split", [2,4,6,8]),
    "min_samples_leaf": hp.choice("min_samples_leaf", [3,4,5]),
    "max_features": hp.choice("max_features", [4,5,6,7])
}


optimization_function = partial(optimize, x = X, y = y)

trials = Trials()

result = fmin(
    algo = tpe.suggest,
    fn = optimization_function,
    space = param_space, 
    max_evals = 20,
    trials = trials
)

print(result)

100%|██████████████████████████████████████████████████████████| 20/20 [34:44<00:00, 104.22s/trial, best loss: -0.7753]
{'criterion': 0, 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 0, 'n_estimators': 0, 'n_jobs': 0}


In [83]:
#### Fitting on the above hyper parameters

In [ ]:
model = ExtraTreesClassifier(max_depth=11, max_features=5, min_samples_leaf=4, min_samples_split=2, n_estimators=100, n_jobs = -1)
# model = ExtraTreesClassifier()

def create_stratified_folds(data, k_fold_num, target_variable):
    data["kfold"] = -1 # we create a new column called kfold and fill it with -1
    data = data.sample(frac=1).reset_index(drop=True) # the next step is to randomize the rows of the data

    y = data[target_variable].values

    kf = model_selection.StratifiedKFold(n_splits= k_fold_num)

    for fold, (trn_, partitioned_idx_) in enumerate(kf.split(X=data, y = y)):
        data.loc[partitioned_idx_, 'kfold'] = fold
    return(data)

# Creating stratified k-fold within the data
k_fold_num = 5

df = create_stratified_folds(df, k_fold_num = 5, target_variable = "Class")

features = [f for f in df.columns if f not in ["kfold","Class"]]

cv_auprc = []
f1_scores = []
for fold in range(k_fold_num):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    x_train = df_train[features].values
    x_valid = df_valid[features].values

    model.fit(x_train, df_train.Class.values)

    valid_preds = model.predict_proba(x_valid)[:, 1]
    valid_preds_binary = model.predict(x_valid)

    auprc = AUPRC(df_valid.Class.values, valid_preds)
    f1 = f1_score(df_valid.Class.values, valid_preds_binary)

    cv_auprc.append(auprc)
    f1_scores.append(np.round(f1,4))

print("Average Area Under Precision Recall Curve = ", np.round(np.mean(cv_auprc),2))
print("Average F1 score = ", np.round(np.mean(f1_scores),2))

In [ ]:
cv_auprc

In [ ]:
f1_scores